Итак, перед тем, как браться за статистику, нужно:

**1. Прочесть исходный файл и превратить его в структуру данных**

К заданию прилагается файл в формате csv, где все значения разделены запятыми. Это наши исходные данные. Чтобы применить к ним все возможности языка Python и библиотеки Pandas, надо импортировать эту библиотеку и сохранить её в переменной. По сокращённому названию панельных данных (panel data), с которых начиналась Pandas, эту переменную принято называть pd:

**import pandas as pd**

Для чтения csv-файла в библиотеке Pandas есть готовая функция — метод **read_csv()**. Как и все методы, он вызывается записью через точку после имени своего объекта. В скобках указывается аргумент (параметр) метода. У read_csv() это имя файла с данными. Прочтение превращает файл в структуру данных DataFrame. Имя переменной, в которой эта структура данных сохраняется, чаще всего df либо отражает тематику данных:

**df = pd.read_csv('music_log.csv')**

**2. Посмотреть на данные**

Вывести на экран таблицу и оценить данные:

print(df)
Как правило, таблица очень велика. Практичнее запросить определённое количество первых строк, методом head().

**3. Оценить качество предподготовки**

Нужно убедиться в том, что данные прошли предподготовку. По крайней мере, не должно быть пропусков и повторов. Пропущенные и неопределённые значения выявляет метод **isna()**, а суммарное количество таких значений — метод **sum()**.

**print(df.isna().sum())**


Повторяющиеся строки — дубликаты — выявляются методом **duplicated()** и подсчитываются тем же sum():

**print(df.duplicated().sum())**
Если возвращаются нули, всё хорошо — данные пригодны для исследования.


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Задача

1. Загрузите таблицу **exoplanet_catalog**, содержащую данные о планетах за пределами Солнечной системы, или экзопланетах. Сформируйте из данной таблицы новый датасет под назанием **exoplanet**, коорая будет содердать стоблцы name, mass, radius, discovered. Проверьте, чтобы название всех столбцов было корректным, при необходмости измените названия.

2. Удалите из получившейся таблица строки с нулевыми значениями.

3. Убедитесь, что таблица не содержит ни пропущенных значений, ни дубликатов.

In [ ]:
#Youre code is here
import pandas as pd

# 1.
db = pd.read_csv('/content/drive/MyDrive/University/IT31/Программирование для анализа данных/Lab3/exoplanet_catalog.csv')

header = ['name', 'mass', 'radius', 'discovered']
exoplanet = pd.DataFrame(data = db, columns = header)
# 2.
exoplanet.fillna(0, inplace = True)
#exoplent.dropna(inplace = True)
exoplanet.drop_duplicates(inplace = True)
exoplanet.reset_index(drop = True, inplace = True)

# 3.
count = exoplanet.duplicated().sum()
print("Количество дубликатов: ", count)

miss = exoplanet.isna().sum()
print("Количество пропущенных значений: \n", miss)


Количество дубликатов:  0
Количество пропущенных значений: 
 name          0
mass          0
radius        0
discovered    0
dtype: int64


# Группировка данных

Слово «анализ» означает разбор, рассмотрение с разных сторон. Анализ данных начинают с разделения их на группы по какому-нибудь признаку. Эта операция называется группировка данных. Она помогает изучить материал более подробно, чтобы затем перейти к поиску взаимосвязей между отдельными группами.


Группировка оправданна, если данные чётко делятся по значимому признаку, а полученные группы близки к теме задачи. Например, когда есть данные обо всех покупках в супермаркете, можно смело заниматься группировкой. Так можно установить время наплыва покупателей и решить проблему пиковых нагрузок. Или посчитать средний чек — обычно для магазинов это ключевая метрика.


Стадии группировки хорошо укладываются в словесную формулу **split-apply-combine**:

•	разделить, **split** — разбиение на группы по определённому критерию;

•	применить, **apply** — применение какого-либо метода к каждой группе в отдельности, например, подсчёт численности группы методом count() или суммирование вызовом sum();

•	объединить, **combine** — сведение результатов в новую структуру данных, в зависимости от условий разделения и выполнения метода это бывает DataFrame и Series.

В библиотеке Pandas есть отличные инструменты группировки. Рассмотрим обращение с ними на примере анализа данных о планетах за пределами Солнечной системы, или экзопланетах. Орбитальные обсерватории засекли уже тысячи таких небесных тел. Их выявляют на снимках космических телескопов наши коллеги, аналитики данных. Поищем среди экзопланет похожие на Землю. Возможно, это наши будущие колонии, или там уже обитают разумные существа, с которыми однажды предстоит установить контакт.

DataFrame с данными по нескольким тысячам экзопланет сохранён в переменной **exoplanet**. Посмотрим на первые 30 строк таблицы:


In [ ]:
import pandas as pd

db = pd.read_csv('/content/drive/MyDrive/University/IT31/Программирование для анализа данных/Lab3/exoplanet_catalog.csv')

header = ['name', 'mass', 'radius', 'discovered']
exoplanet = pd.DataFrame(data = db, columns = header)
exoplanet.head(30)

,name,mass,radius,discovered
0,NaN,NaN,NaN,2008.0
1,NaN,21.0,NaN,2007.0
2,NaN,NaN,NaN,2009.0
3,NaN,NaN,NaN,2008.0
4,NaN,NaN,NaN,2002.0
5,NaN,5.8,NaN,2006.0
6,NaN,NaN,NaN,1996.0
7,NaN,NaN,NaN,2020.0
8,NaN,NaN,NaN,2008.0
9,NaN,NaN,0.000002,2017.0


**Документация**

Столбцы:

•	name: название экзопланеты;

•	mass: масса в массах планеты Юпитер;

•	radius: радиус, пересчитанный в радиусах Земли;

•	discovered: год открытия экзопланеты.

*Источник: каталог экзопланет на портале exoplanet.eu*

На картинке изображен принцип **split-apply-combine** для таблицы с экзопланетами. Посмотрим, как вообще идут дела с поиском экзопланет. Сначала данные делят по группам, где каждая группа — это год. Потом метод **count()** подсчитывает численность каждой группы. В итоге получаем новую структуру данных с группами, где каждая содержит год и число открытых за этот год экзопланет.

![image.png](attachment:image.png)

В Рandas для группировки данных есть метод **groupby()**. Он принимает как аргумент название столбца, по которому нужно группировать. В случае с делением экзопланет по годам открытия:


**print(exoplanet.groupby('discovered'))**

**<pandas.core.groupby.DataFrameGroupBy object at 0x7fc1e1ca3400>**


Применение метода **groupby()** к объекту типа DataFrame приводит к созданию объекта особого типа — **DataFrameGroupBy**. Это сгруппированные данные. Если применить к ним какой-нибудь метод Pandas, они станут новой структурой данных типа **DataFrame** или **Series**.
Подсчитаем сгруппированные по годам экзопланеты методом **count()**:


In [ ]:
#Youre code is here
exoplanet.groupby('discovered').count()

,name,mass,radius
discovered,,,
1988.0,0,1,0
1992.0,0,3,0
1995.0,0,3,1
1996.0,0,4,1
1998.0,0,1,0
1999.0,0,6,2
2000.0,0,11,5
2001.0,0,6,1
2002.0,0,8,4


Если нужно сравнить наблюдения по одному показателю, метод применяют к **DataFrameGroupBy** с указанием на один столбец. Нас в первую очередь интересует радиус экзопланет: мы ищем другую Землю. Давайте получим таблицу с единственным столбцом 'radius':

**exo_number = exoplanet.groupby('discovered')['radius'].count()**

**print(exo_number)**

In [ ]:
#Youre code is here
exo_number = exoplanet.groupby('discovered')['radius'].count()

exo_number


,radius
discovered,
1988.0,0
1992.0,0
1995.0,1
1996.0,1
1998.0,0
1999.0,2
2000.0,5
2001.0,1
2002.0,4


Получили Series, где по годам открытия расписано количество экзопланет, для которых удалось установить радиус.

Посмотрим, как меняется средний радиус открытых экзопланет год от года. Для этого надо сложить радиусы планет, открытых за определённый год, и поделить на их количество (которое мы уже нашли).

Сумма радиусов считается методом **sum()**:

**exo_radius_sum = exoplanet.groupby('discovered')['radius'].sum()**


**print(exo_radius_sum)**



In [ ]:
#Youre code is here
exo_radius_sum = exoplanet.groupby('discovered')['radius'].sum()

exo_radius_sum

,radius
discovered,
1988.0,0.000000
1992.0,0.000000
1995.0,1.900000
1996.0,1.060000
1998.0,0.000000
1999.0,2.410000
2000.0,5.107000
2001.0,0.921000
2002.0,4.210000


Очень кстати, что объекты Series можно делить друг на друга. Это позволит нам разделить перечень сумм радиусов на перечень количеств экзопланет без перебора в цикле:


**exo_radius_mean = exo_radius_sum/exo_number**

**print(exo_radius_mean)**


In [ ]:
#Youre code is here
exo_radius_mean = exo_radius_sum/exo_number

exo_radius_mean

,radius
discovered,
1988.0,NaN
1992.0,NaN
1995.0,1.900000
1996.0,1.060000
1998.0,NaN
1999.0,1.205000
2000.0,1.021400
2001.0,0.921000
2002.0,1.052500


Точность наших приборов растёт, и новые экзопланеты по размерам всё ближе к Земле.

А теперь вернемся к анализу датафрейма с музыкльными предпочтениями слушателей Яндекс.Музыки. Идею объединения сервисов Музыка и Радио тестировали на небольшой группе пользователей. Результаты сведены в csv-файл, который вам предстоит изучить. Итог анализа таких данных — это метрики: величины, значения которых отражают пользовательские впечатления. Одна из важнейших — **happiness**. Здесь это среднее время, которое пользователь слушает музыку в течение выбранного периода времени (в нашей задаче — за сутки). Чем дольше пользователь слушает музыку, тем он довольнее. Ваша задача: найти значение happiness и посмотреть, как оно менялось.

Тем же методом **groupby()**, которым мы ищем новую Землю, можно поискать и необыкновенного человека в данных Яндекс.Музыки. Тем более, что без этого не выполнить поставленной менеджером задачи.

Прежде, чем рассчитывать метрику happiness, нужно изучить пользователей, чьё «счастье» мы собираемся оценить. Какие они, эти люди, которые слушают действительно много музыки? Есть ли у них особые предпочтения, или они потребляют всё подряд?


## Задача

1. Меломаны у нас есть. Сейчас узнаем идентификатор **user_id** одного из них. Для этого сгруппируем данные по каждому пользователю, чтобы собрать жанры прослушанных им композиций.

Сгруппируйте DataFrame по столбцу user_id, сохраните полученный результат в переменной genre_grouping.

Посчитайте количество жанров, которые выбрали пользователи, методом count(), указав, что выбираем один столбец genre_name.

Сохраните результат в переменной genre_counting и выведите первые 30 строк этой таблицы.

2. Быть может, те, кто за день слушает больше 50 песен, имеют более широкие предпочтения. Чтобы найти такого, изготовим универсальный инструмент.

Напишите функцию user_genres, которая принимает некую группировку как свой аргумент group. Функция должна перебирать группы, входящие в эту группировку.

В каждой группе два элемента — имя группы с индексом 0 и список значений с индексом 1.

Обнаружив такую группу, в которой список (элемент с индексом 1) содержит более 50 значений, функция возвращает имя группы (значение элемента с индексом 0).

3. Вызовите функцию **user_genres**, как аргумент передайте ей **genre_grouping**. Результат – **user_id** неведомого нам любителя музыки – сохраните в переменной **search_id** и выведите значение на экран.


In [3]:
import pandas as pd

# Загрузка данных из CSV файла
db = pd.read_csv('/content/drive/MyDrive/University/IT31/Программирование для анализа данных/Lab3/yandex_music.csv')
new_names = ['user_id', 'track_name', 'artist_name', 'genre_name', 'total_play_seconds', 'time_name', 'day_name']
new_table = db.set_axis(new_names, axis='columns')

# 1. Сгруппируем данные по user_id и соберем жанры прослушанных композиций
genre_grouping = new_table.groupby('user_id')['genre_name']
genre_counting = genre_grouping.count()
print(genre_counting.head(30))

# 2. Функция для поиска user_id пользователя, который слушает более 50 песен
def user_genres(group):
    for g in group:
        if len(g[1]) > 50:
            return g[0]
    return None

# 3. Вызовем функцию user_genres, передав ей genre_grouping

search_id = user_genres(list(genre_grouping))
print(f"Идентификатор меломана: {search_id}")



user_id
10004220    3
100173ED    1
1001CE90    7
1002410A    3
100261F9    2
1004AAE6    1
1004C5D8    1
1005BECA    1
10089377    3
100AB021    1
100ACFBC    1
10101201    2
10107FC7    2
10112F4C    2
1016144E    1
1016C39F    2
10180175    1
101861D1    2
10189D10    8
101AAA19    1
101B046A    2
101B53F1    1
101BBC61    1
101C12A4    2
101D714C    1
101E910F    1
10211433    1
10222F55    1
1022B77E    1
10246DCB    2
Name: genre_name, dtype: int64
Идентификатор меломана: 414F229D


# Сортировка данных

Поиск необычного в группе — что среди планет, что среди меломанов — это прежде всего поиск чемпионов: объектов с выдающимися показателями по разным статьям. Как всю таблицу, так и отдельные группы изучают, сортируя строки по какому-либо столбцу.

В Pandas для этой операции есть метод **sort_values()**. У него два аргумента:

• **by = 'имя столбца'** — имя столбца, по которому нужно сортировать;

• **ascending:** по умолчанию True. Для сортировки по убыванию установите значение False.

![image.png](attachment:image.png)


Среди экзопланет интересны близкие по размерам к Земле. Есть ли такие? Отсортируем список по радиусу в порядке возрастания. Тогда в голове таблицы окажутся самые малые, на которых гравитация не прижмёт нас к полу.

**print(exoplanet.sort_values(by='radius').head(30))**


In [ ]:
import pandas as pd

db = pd.read_csv('/content/drive/MyDrive/University/IT31/Программирование для анализа данных/Lab3/exoplanet_catalog.csv')
header = ['name', 'mass', 'radius', 'discovered']
exoplanet = pd.DataFrame(data = db, columns = header)

print(exoplanet.sort_values(by='radius').head(30))


      name     mass    radius  discovered
9      NaN      NaN  0.000002      2017.0
16     NaN      NaN  0.000062      2019.0
4357   NaN      NaN  0.000900      2019.0
3258   NaN      NaN  0.025430      2014.0
3207   NaN  0.01000  0.026000      2013.0
3354   NaN      NaN  0.035950      2015.0
1744   NaN  0.01353  0.042000      2014.0
3355   NaN      NaN  0.044000      2015.0
2126   NaN      NaN  0.046000      2016.0
2221   NaN  0.00021  0.047000      2014.0
3356   NaN      NaN  0.047000      2015.0
3611   NaN  0.01300  0.048000      2013.0
3357   NaN      NaN  0.049000      2015.0
3323   NaN  0.00300  0.051000      2011.0
1745   NaN  0.00944  0.052000      2014.0
2491   NaN      NaN  0.054000      2016.0
1839   NaN      NaN  0.054000      2016.0
1611   NaN      NaN  0.055000      2016.0
2212   NaN      NaN  0.057000      2016.0
2185   NaN      NaN  0.058000      2016.0
2439   NaN      NaN  0.058000      2016.0
2937   NaN      NaN  0.059000      2016.0
3337   NaN      NaN  0.060000     

Оказывается, некоторые из уже открытых экзопланет по размерам близки не то что к Земле, но уже и к Луне! Получим список экзопланет с радиусом меньше земного. Смотрите, как логический оператор (здесь это <) задействуется в отборе элементов столбца. Дальше нам этот приём не раз понадобится.


**print(exoplanet[exoplanet['radius'] < 1])**

In [ ]:
#Youre code is here
print(exoplanet[exoplanet['radius'] < 1])

      name      mass    radius  discovered
9      NaN       NaN  0.000002      2017.0
16     NaN       NaN  0.000062      2019.0
22     NaN  20.00000  0.920000      2010.0
41     NaN  43.00000  0.910000      2021.0
42     NaN  67.00000  0.970000      2021.0
...    ...       ...       ...         ...
4731   NaN       NaN  0.181110      2018.0
4732   NaN  26.00000  0.920000      2009.0
4734   NaN   0.62000  0.973000      2007.0
4751   NaN  59.30000  0.783000      2021.0
4783   NaN   0.01517  0.167190      2018.0

[2974 rows x 4 columns]


Но и этот список такой длинный, что изучать его лучше по частям. Экзопланеты, близкие по размерам к Земле, за последнее десятилетие открывали нередко. Можно изучать список открытых за каждый год. Например, для 2014 года (вновь обратите внимание на работу логического оператора, теперь это ==):

**print(exoplanet[exoplanet['discovered'] == 2014])**


In [ ]:
#Youre code is here
print(exoplanet[exoplanet['discovered'] == 2014])

      name  mass  radius  discovered
106    NaN   NaN     NaN      2014.0
222    NaN   NaN     NaN      2014.0
223    NaN   NaN     NaN      2014.0
233    NaN   NaN     NaN      2014.0
234    NaN   NaN     NaN      2014.0
...    ...   ...     ...         ...
4763   NaN   NaN     NaN      2014.0
4764   NaN   NaN     NaN      2014.0
4776   NaN   NaN     NaN      2014.0
4784   NaN   NaN     NaN      2014.0
4795   NaN  40.0     NaN      2014.0

[872 rows x 4 columns]


А чтобы не тратить время на лишнее, поставим оба условия сразу. Для этого в Pandas есть логический оператор &, подобный оператору and языка Python. Напомним, его смысл на русском языке можно передать словами «и ещё»:

**# экзопланеты меньше Земли __ и ещё __ открытые в 2014 году**

**exo_small_14 = exoplanet[ (exoplanet['radius']<1) & (exoplanet['discovered']==2014)]**

**print(exo_small_14)**



In [ ]:
#Youre code is here
exo_small_14 = exoplanet[((exoplanet['radius'] < 1) & (exoplanet['discovered'] == 2014))]
print(exo_small_14)

      name    mass  radius  discovered
400    NaN  0.7600   0.944      2014.0
465    NaN  0.2370   0.912      2014.0
477    NaN  0.3190   0.998      2014.0
1164   NaN  0.0372   0.226      2014.0
1690   NaN  1.4500   0.940      2014.0
...    ...     ...     ...         ...
4039   NaN  0.0410   0.305      2014.0
4051   NaN  0.0270   0.238      2014.0
4065   NaN  0.0110   0.132      2014.0
4077   NaN  0.0110   0.178      2014.0
4088   NaN  0.0190   0.132      2014.0

[769 rows x 4 columns]


Отсортируем результат в порядке убывания радиуса.

**print(exo_small_14.sort_values(by = 'radius', ascending = False))**


In [ ]:
#Youre code is here
print(exo_small_14.sort_values(by='radius', ascending=False))

      name     mass   radius  discovered
477    NaN  0.31900  0.99800      2014.0
3329   NaN  0.25000  0.97800      2014.0
3319   NaN  2.50000  0.96000      2014.0
400    NaN  0.76000  0.94400      2014.0
1690   NaN  1.45000  0.94000      2014.0
...    ...      ...      ...         ...
2263   NaN      NaN  0.06200      2014.0
1745   NaN  0.00944  0.05200      2014.0
2221   NaN  0.00021  0.04700      2014.0
1744   NaN  0.01353  0.04200      2014.0
3258   NaN      NaN  0.02543      2014.0

[769 rows x 4 columns]


Самая крупная планета, Kepler 106 d – почти как Земля, вращается вокруг звезды Kepler 106 в созвездии Лебедя. Эта звезда очень похожа на наше Солнце. Правда, до неё 1435 световых лет — далековато. Но, возможно, учёные что-нибудь придумают. А мы пока применим эту технологию к нашему бизнесу, в «приземлённой» задаче.
## Задача

1. Космический телескоп Kepler открыл похожую на Землю планету у похожей на Солнце звезды. А вы в данных Яндекс.Музыки обнаружили меломана с уникальными данными. Он за день послушал больше 50 композиций.
Получите таблицу с прослушанными им треками.

Для этого запросите из структуры данных df строки, отвечающие сразу двум условиям:

1) значение в столбце 'user_id' должно быть равно значению переменной search_id;

2) время прослушивания, т.е. значение в столбце 'total_play_seconds', не должно равняться 0.

Сохраните результат в переменной music_user.

2. Теперь узнаем, сколько времени он слушал музыку каждого жанра.

Сгруппируйте данные таблицы music_user по столбцу 'genre_name' и получите сумму значений столбца 'total_play_seconds'. Сохраните результат в переменной sum_music_user и выведите её значение на экран.

3. Кажется, предпочтения нашего меломана начинают проявляться. Но, возможно, длительность композиций от жанра к жанру сильно различается. Важно знать, сколько треков каждого жанра он включил.

Сгруппируйте данные по столбцу genre_name и посчитайте, сколько значений в столбце genre_name. Сохраните результат в переменной count_music_user и выведите её значение на экран.

4. Чтобы предпочтения были видны сразу, нужно крупнейшие значения расположить наверху. Отсортируйте данные в группировке sum_music_user по убыванию. Внимание: когда применяете метод sort_values() к Series с единственным столбцом, аргумент by указывать не нужно, только порядок сортировки.

Сохраните результат в переменной final_sum и выведите её значение на экран.

5. Теперь то же самое надо сделать с числом прослушанных меломаном композиций. Отсортируйте данные группировки count_music_user по убыванию. Сохраните результат в переменной final_count, значение которой выведите на экран.


In [12]:
#Продолжение предыдущего кода
import pandas as pd

# Загрузка данных из CSV файла
db = pd.read_csv('/content/drive/MyDrive/University/IT31/Программирование для анализа данных/Lab3/yandex_music.csv')
new_names = ['user_id', 'track_name', 'artist_name', 'genre_name', 'total_play_seconds', 'time_name', 'day_name']
new_table = db.set_axis(new_names, axis='columns')

# Сгруппируем данные по user_id и соберем жанры прослушанных композиций
genre_grouping = new_table.groupby('user_id')['genre_name']
genre_counting = genre_grouping.count()
#print(genre_counting.head(30))

# Функция для поиска user_id пользователя, который слушает более 50 песен
def user_genres(group):
    for g in group:
        if len(g[1]) > 50:  # Проверяем, содержит ли группа больше 50 значений
            return g[0]     # Возвращаем имя группы (user_id)
    return None

# Вызовем функцию user_genres, передав ей genre_grouping
search_id = user_genres(list(genre_grouping))
print(f"Идентификатор меломана: {search_id}\n")
#--------------------------------
# 1.
music_user = new_table[(new_table['user_id'] == search_id) & (new_table['total_play_seconds'] != 0)]
# 2.
sum_music_user = new_table.groupby('genre_name')['total_play_seconds'].count()
print(f"Сумма прослушанных треков каждого жанра: \n{sum_music_user}\n")
# 3.
count_music_user = new_table.groupby('genre_name')['genre_name'].count()
print(f"Количество треков, кажого жанра: \n{count_music_user}\n ")
# 4.
final_sum = sum_music_user.sort_values(ascending=False)
print(f"Сумма прослушанных треков каждого жанра: \n{final_sum}\n")
# 5.
final_count = count_music_user.sort_values(ascending=False)
print(f"Количество треков, кажого жанра: \n{final_count}\n")

# Сколько человек слушает поп-музыку из Санкт-Петербурга
count_people = new_table[((new_table['genre_name'] == 'pop') & (new_table['total_play_seconds'] == 'Saint-Petersburg'))]
print(f"\n\nКоличество человек, слушающих поп-музыку в Санкт-Петербурге: {count_people.count()}")

Идентификатор меломана: 414F229D

Сумма прослушанных треков каждого жанра: 
genre_name
acid              1
acoustic          6
action            4
adult            27
africa           17
               ... 
western         106
world          2069
worldbeat         2
ïîï               1
электроника       1
Name: total_play_seconds, Length: 289, dtype: int64

Количество треков, кажого жанра: 
genre_name
acid              1
acoustic          6
action            4
adult            27
africa           17
               ... 
western         106
world          2069
worldbeat         2
ïîï               1
электроника       1
Name: genre_name, Length: 289, dtype: int64
 
Сумма прослушанных треков каждого жанра: 
genre_name
pop                 8850
dance               6761
rock                6192
electronic          5852
hip                 3148
                    ... 
lovers                 1
loungeelectronic       1
laiko                  1
kayokyoku              1
электроника            1
N